In [2]:
import mlflow
import mlflow.sklearn
from datetime import datetime

### 🔹 Cell 2: Set Experiment Name

In [5]:
mlflow.set_tracking_uri("http://127.0.0.1:8080")
mlflow.set_experiment("Lab3_Model_signature")


2025/10/05 19:46:13 INFO mlflow.tracking.fluent: Experiment with name 'Lab3_Model_signature' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/659266294290654579', creation_time=1759668373373, experiment_id='659266294290654579', last_update_time=1759668373373, lifecycle_stage='active', name='Lab3_Model_signature', tags={}>

In [14]:
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd


# ---------- Config ----------
TRACKING_URI = "http://127.0.0.1:8081"
EXPERIMENT_NAME = "Lab_MLflow_Tracking_Quickstart"
REGISTERED_MODEL_NAME = "iris_model_tracking_quickstart"


# ---------- Connect to Tracking ----------
mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)


# ---------- Data ----------
X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# ---------- Train ----------
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train, y_train)


# ---------- Metrics ----------
train_pred = lr.predict(X_train)
train_acc = accuracy_score(y_train, train_pred)

# ✨ Add test metrics too
test_pred = lr.predict(X_test)
test_acc = accuracy_score(y_test, test_pred)



# ---------- Signature & Example ----------
signature = infer_signature(X_train, train_pred)
input_example = X_train.head(1)

In [15]:
signature

inputs: 
  ['sepal length (cm)': double (required), 'sepal width (cm)': double (required), 'petal length (cm)': double (required), 'petal width (cm)': double (required)]
outputs: 
  [Tensor('int64', (-1,))]
params: 
  None

In [16]:
input_example

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
8,4.4,2.9,1.4,0.2


In [ ]:


params = {
    "model": "logreg", 
    "max_iter": 1000, 
    "solver": lr.solver,
    "random_state": 42  # ✨ Track for reproducibility
}


with mlflow.start_run(run_name="train-logreg-iris") as run:
    # log params/metrics
    mlflow.log_params(params)
    mlflow.log_metric("train_accuracy", train_acc)  # ✨ Renamed for clarity
    mlflow.log_metric("test_accuracy", test_acc)    # ✨ Added test metric
    
    # ✨ Optional: Log dataset info
    mlflow.log_param("train_size", len(X_train))
    mlflow.log_param("test_size", len(X_test))
    
    # log & register model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="model",
        signature=signature,
        input_example=input_example,
        registered_model_name=REGISTERED_MODEL_NAME,
        pip_requirements=["scikit-learn", "mlflow"],
    )
    
    print("Run ID:", run.info.run_id)
    print("Model URI:", model_info.model_uri)
    print(f"Train Accuracy: {train_acc:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")

2025/10/06 09:20:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'iris_model_tracking_quickstart' already exists. Creating a new version of this model...
2025/10/06 09:20:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris_model_tracking_quickstart, version 2
Created version '2' of model 'iris_model_tracking_quickstart'.


Run ID: 37a8cc39dd5d4870b43da7d28884934f
Model URI: models:/m-49eb3b42ec2340798c1da5304b58d546
Train Accuracy: 0.9750
Test Accuracy: 0.9667
🏃 View run train-logreg-iris at: http://127.0.0.1:8081/#/experiments/451274039144840475/runs/37a8cc39dd5d4870b43da7d28884934f
🧪 View experiment at: http://127.0.0.1:8081/#/experiments/451274039144840475


In [8]:
print(model_info.model_uri)

models:/m-49eb3b42ec2340798c1da5304b58d546


In [9]:
run.info.run_id

'37a8cc39dd5d4870b43da7d28884934f'

In [ ]:

# ---------- Verify model_uri exists ----------
from mlflow.tracking import MlflowClient
client = MlflowClient()
artifacts = client.list_artifacts(run_id=run.info.run_id, path="model")
print("Artifacts under 'model/' in this run:", [a.path for a in artifacts])

Artifacts under 'model/' in this run: ['modelx/MLmodel', 'modelx/conda.yaml', 'modelx/input_example.json', 'modelx/model.pkl', 'modelx/python_env.yaml', 'modelx/requirements.txt', 'modelx/serving_input_example.json']


In [13]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
y_pred = loaded_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test accuracy from loaded model: {acc:.4f}")

Test accuracy from loaded model: 0.9667
